In [1]:
import pandas as pd
import json
import datetime
import numpy as np

In [2]:
# Depending on the current year, we want a specified 10 year range to report data on
years = [datetime.datetime.now().year - 2, datetime.datetime.now().year + 8]

# Load up config
with open('config.json', 'r') as file:
    data = json.load(file)

# Define paths dict
paths_dict = data["file_paths"]

In [3]:
hot_50_input = paths_dict["HOT_50_INPUT"]["path"]
hot_50_output = paths_dict["HOT_50_OUTPUT"]["path"]


In [ ]:
!python hot_50_pdf_scraper.py -i {hot_50_input} -c {hot_50_output}

## Read in Data

In [4]:
hot50 = pd.read_csv(paths_dict["HOT_50_OUTPUT"]["path"])
valid_majors = pd.read_csv(paths_dict["MAJORS"]["path"], dtype=str)
crosswalk = pd.read_excel(paths_dict["CIP_TO_SOC_CROSSWALK"]["path"], dtype=str)
mismatched_codes = pd.read_csv(paths_dict["MISMATCHED_LIGHTCAST_CODES"]["path"])

## Match CIP to SOC with OU Majors so we can obtain SOCS for Majors

In [5]:
## Codes and titles have years associated with them, we want them to be generalized so we don't have to rename the columns every time
## we want to run the scripting
CIP_CODE_TAG = crosswalk.columns[0]
CIP_TITLE = crosswalk.columns[1]
SOC_CODE_TAG = crosswalk.columns[2]
SOC_TITLE = crosswalk.columns[3]

crosswalk = crosswalk.rename(columns={CIP_CODE_TAG: "CIP_CODE_TAG", CIP_TITLE: "CIP_TITLE",
                          SOC_CODE_TAG: "SOC_CODE_TAG", SOC_TITLE: "SOC_TITLE"})

# We want a "hot job" label for those on the hot jobs list
hot50["Hot Tag"] = "T"

# Get crosswalk ready for merge
crosswalk["CIP_CODE_TAG"] = crosswalk["CIP_CODE_TAG"].str.replace(".", "-")

# Get valid majors ready for merge
valid_majors["stvmajr_cipc_code"] = valid_majors["stvmajr_cipc_code"].str.replace('.0', '')
valid_majors["stvmajr_cipc_code"] = valid_majors["stvmajr_cipc_code"].str[:-4] + '-' + valid_majors["stvmajr_cipc_code"].str[-4:]

In [6]:
# Merge together Oakland Majors with Crosswalk information and include a tag if it's a part of the hot jobs
ou_majors_with_soc = pd.merge(valid_majors, crosswalk, left_on=["stvmajr_cipc_code"], right_on=["CIP_CODE_TAG"], how="left")
ou_majors_with_soc = pd.merge(ou_majors_with_soc, hot50, left_on=["SOC_TITLE"], right_on=["Profession"], how="left")

In [7]:
# We only care about specific columns and those that have SOC codes
ou_majors_with_soc = ou_majors_with_soc.loc[(ou_majors_with_soc["SOC_CODE_TAG"].notnull()), ["stvmajr_code", "stvmajr_desc", "CIP_CODE_TAG", "CIP_TITLE", "SOC_CODE_TAG", "SOC_TITLE", "Hot Tag"]]

In [8]:
# Grab distinct code list
codes = ou_majors_with_soc["SOC_CODE_TAG"].dropna().drop_duplicates().to_list()

# Write codes into a txt file
with open(paths_dict["SOC_CODES_FOR_ONET"]["path"], "w") as file:
    file.writelines(code + '\n' for code in codes)

## Obtain ONET data for each of our SOC codes vs ONET API

In [ ]:
!python ../ONET/ONET_API.py < Inputs/SOCcodes.txt > Outputs/ONET.json

##### Cleaning ONET Data

In [9]:
# Open up the ONET json dictionary
with open(f"Outputs/ONET.json", "r") as file:
    ONET_data = json.load(file)

# Move dictionary to a dataframe where the different SOCs are rows
ONET_df = pd.DataFrame.from_dict(ONET_data, orient='index')

In [10]:
# Expand "results" columns so they are columns in the df
ONET_df = ONET_df["results"].apply(pd.Series)

In [11]:
# Quick function for cleaning one of the results columns
def get_MI_quotient(row):
    try:
        for category in row.keys():
            for state in row[category]['state']:
                if state["name"] == "Michigan":
                    return state["location_quotient"]  
        return None
    except:
        return None

In [12]:
# Take desired values out of the dictionaries in their current columns
ONET_df["abilities"] = ONET_df["abilities"].apply(lambda x: ', '.join(group['title']['name'] for group in x['group']) if isinstance(x, dict) else None)
ONET_df["aliases"] = ONET_df["career"].apply(lambda x: ', '.join(x["also_called"]["title"]) if isinstance(x, dict) and "also_called" in x and "title" in x["also_called"] else None)
ONET_df["task"] = ONET_df["career"].apply(lambda x: x["what_they_do"] if isinstance(x, dict) and "what_they_do" in x else None)
ONET_df["knowledge"] = ONET_df["knowledge"].apply(lambda x: ', '.join(group['title']['name'] for group in x['group']) if isinstance(x, dict) else None)
ONET_df["personality_type"] = ONET_df["personality"].apply(lambda x: x["top_interest"]["title"])
ONET_df["interests"] = ONET_df["personality"].apply(lambda x: x["top_interest"]["description"])
ONET_df["skills"] = ONET_df["skills"].apply(lambda x: ', '.join(group['title']['name'] for group in x['group']) if isinstance(x, dict) else None)
ONET_df["technology"] = ONET_df["technology"].apply(lambda x: ', '.join(group['title']['name'] for group in x['category']) if isinstance(x, dict) else None)
ONET_df["where_do_they_work"] = ONET_df["where_do_they_work"].apply(lambda x: ', '.join(f"{group['title']} ({group['percent_employed']}%)" for group in x["industry"]) if isinstance(x, dict) else None)
ONET_df["MI_to_rel_avg"] = ONET_df["check_out_my_state"].apply(lambda x: get_MI_quotient(x))
ONET_df["education"] = ONET_df["education"].apply(
    lambda x: ', '.join([word.title() for word in x["education_usually_needed"]["category"]]) 
    if "education_usually_needed" in x and x["education_usually_needed"].get("category") else ''
)


In [13]:
# Filter specific ONET columns
ONET_df = ONET_df.loc[:, ["abilities", "aliases", "task", "knowledge", "personality_type", "interests", "skills", "technology", "where_do_they_work", "MI_to_rel_avg", "education"]]


In [14]:
# Set SOC to a column and remove index
ONET_df.index.names = ["SOC"]
ONET_df.reset_index(inplace=True)

# Remove ".00" from SOC Codes
ONET_df["SOC"] = ONET_df["SOC"].str[:-3]

In [15]:
# We also want to account for codes that were incorrect in lightcast, so we match with the mismatches and change them to update to our correct classifications
ONET_df = pd.merge(ONET_df, mismatched_codes, left_on="SOC", right_on="Crosswalk", how="left")
ONET_df["SOC"] = np.where(ONET_df["OEWS"].notnull(), ONET_df["OEWS"], ONET_df["SOC"])

In [16]:
ONET_df.to_excel(paths_dict["ONET_INFO_FOR_SOCS"]["path"])

## Matching ONET Data and LightCast Data to OU Data


In [17]:
# Read in lightcast data
lightcast = pd.read_excel(paths_dict["LIGHTCAST_OCCUPATION_REPORT"]["path"], sheet_name="Occs")

# Merge ONET and lightcast data together on SOC code
lightcast_code_matched = pd.merge(ONET_df, lightcast, on=["SOC"], how="left")

In [18]:
# We also want to account for codes that were incorrect in lightcast, so we match with the mismatches and change them to update to our correct classifications
ou_majors_with_soc = pd.merge(ou_majors_with_soc, mismatched_codes, left_on="SOC_CODE_TAG", right_on="Crosswalk", how="left")
ou_majors_with_soc["SOC_CODE_TAG"] = np.where(ou_majors_with_soc["OEWS"].notnull(), ou_majors_with_soc["OEWS"], ou_majors_with_soc["SOC_CODE_TAG"])

In [19]:
# Merge with OU Data
lightcast_ou_matched = pd.merge(ou_majors_with_soc, lightcast_code_matched, right_on=["SOC"], left_on=["SOC_CODE_TAG"], how="left")

lightcast_ou_matched = lightcast_ou_matched.drop_duplicates(subset=["stvmajr_desc", "SOC_TITLE"], keep="first")

In [20]:
# Create df for Jobs Increase visualization
ou_all_data_jobs = lightcast_ou_matched.melt(id_vars=["CIP_CODE_TAG", "stvmajr_desc", "SOC_TITLE"], value_vars=[f"{years[0]} Jobs", f"{years[1]} Jobs"], var_name="Jobs Year", value_name="Jobs")
ou_all_data_wages = lightcast_ou_matched.melt(id_vars=["CIP_CODE_TAG", "stvmajr_desc", "SOC_TITLE"], value_vars=["Pct. 10 Annual Earnings", "Pct. 25 Annual Earnings", "Median Annual Earnings", "Pct. 75 Annual Earnings", "Pct. 90 Annual Earnings"], var_name="PCT Category", value_name="Earnings")
ou_all_data_demographics = lightcast_ou_matched.melt(id_vars=["CIP_CODE_TAG", "stvmajr_desc", "SOC_TITLE"], value_vars=["Current Year Total Diversity % of Occupation", "Current Year White % of Occupation", "Current Year Males % of Occupation", "Current Year Females % of Occupation", "Current Year Age 14-18 % of Occupation", "Current Year Age 19-21 % of Occupation", "Current Year Age 22-24 % of Occupation", "Current Year Age 25-34 % of Occupation", "Current Year Age 35-44 % of Occupation", "Current Year Age 45-54 % of Occupation", "Current Year Age 45-54 % of Occupation", "Current Year Age 55-64 % of Occupation", "Current Year Age 65+ % of Occupation"], var_name="Demographic", value_name="PCT Demographic")

# Change Variable Name Values to be Usable in Looker
ou_all_data_jobs["Jobs Year"] = ou_all_data_jobs["Jobs Year"].str[:4]

# Lightcast puts filtered year columns in the df, we want to rename them to be generalizable
lightcast_ou_matched = lightcast_ou_matched.rename(columns={f"{years[0]} Jobs": "Beginning Range Jobs", f"{years[1]} Jobs": "End Range Jobs", 
                                                            f"{years[0]} - {years[1]} Change": "Jobs Range Change", f"{years[0]} - {years[1]} % Change": "Jobs Range PCT Change",
                                                            f"{years[0]} - {years[1]} Openings": "Jobs Range Openings"})



ou_all_data_wages.loc[ou_all_data_wages["PCT Category"] == "Median Annual Earnings", "PCT Category"] = "Pct. 50 Annual Earnings"
ou_all_data_wages["PCT Category"] = ou_all_data_wages["PCT Category"].str[5:7]

In [21]:
# Drop dups
ou_all_data_jobs = ou_all_data_jobs.drop_duplicates(keep="first")
ou_all_data_wages = ou_all_data_wages.drop_duplicates(keep="first")
ou_all_data_demographics = ou_all_data_demographics.drop_duplicates(keep="first")

In [22]:
# Save dfs
lightcast_ou_matched.to_excel(paths_dict["PROGRAM_STATS"]["path"], index=False)
ou_all_data_jobs.to_excel(paths_dict["JOB_COUNTS"]["path"], index=False)
ou_all_data_wages.to_excel(paths_dict["JOB_WAGES"]["path"], index=False)
ou_all_data_demographics.to_excel(paths_dict["JOB_DEMOGRAPHICS"]["path"], index=False)

## Take Manual Job Posting Information from Lightcast

In [23]:
codes = lightcast_ou_matched["SOC"].dropna().drop_duplicates().tolist()

In [24]:
import os

gen_path = paths_dict["GOOGLE_FOLDER_MI_HIRING"]["path"]
dataframes = []

for code in codes:
    new_path = os.path.join(gen_path, f"{code}.csv")
    try:
        tmp = pd.read_csv(new_path).iloc[0:50, :]
    except:
        continue
    tmp["SOC"] = code
    dataframes.append(tmp)

agg = pd.concat(dataframes, ignore_index=True)

In [25]:
agg = agg[agg["Latest 365 Days Unique Postings"] != 0]

In [26]:
agg.to_csv(paths_dict["LIGHTCAST_MI_JOBS"]["path"])